In [3]:
!export HF_HOME=/scratch/gilbreth/dparveez/

In [4]:
BASE_PATH = '/scratch/gilbreth/dparveez/'
MODEL_NAME = "gpt2-xl"

In [6]:
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

torch.cuda.empty_cache()

In [7]:
device = torch.device(f'cuda:0')

model = AutoModelForCausalLM.from_pretrained(os.path.join(BASE_PATH, "fine_tuned_model/")).to(device)
tokenizer = AutoTokenizer.from_pretrained(os.path.join(BASE_PATH, "fine_tuned_model/"))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token
model.config._name_or_path='gpt2-xl'

In [91]:
def generate_response(sentence):
    # Define the prompt template
    prompt_template = (
        "You are a helpful chatbot for Purdue University Fort Wayne's Computer Science Department. "
        "You will answer questions about courses and faculty in the CS department at Purdue Univeristy Fort Wayne (PFW) ONLY. "
        "Be as concise as possible. If you are unsure about a quesiton, reply with something along the lines of 'I cannot help you with that'. "
        f"\nQ: {sentence}"
        "\n"
    )
    
    # print(prompt_template)

    inputs = tokenizer.encode(prompt_template, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=125,  # Increased max length to accommodate template + answer
            num_return_sequences=1,
            no_repeat_ngram_size=3,
            top_k=50,
            top_p=0.85,
            temperature=0.2,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    q_idx = response.find("Q:")
    end_idx = response[len(prompt_template):].find("\n")
    
    if end_idx == -1:
        end_idx = len(response)
    else:
        end_idx = len(prompt_template) + end_idx
    
    return response[q_idx: end_idx]

In [92]:
print(generate_response("What does Professor Jon Rusert teach?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What does Professor Jon Rusert teach?
A: Professor JonRusert teaches courses including Computer Architecture, Intro to C and UNIX, Natural Language Processing, Topics in Advanced NLP, and Research MS Thesis. He also teaches Python


In [93]:
print(generate_response("Who teaches advanced NLP concepts at PFW?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Who teaches advanced NLP concepts at PFW?
A: Advanced Topics in NLP courses are taught by Professor Jon Rusert.


In [94]:
print(generate_response("Who is the Chair of CS?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Who is the Chair of CS?
A: The Chair of the Computer Science department is Adolfo Coronado.


In [99]:
print(generate_response("What are some advanced topics in Computer Graphics taught?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What are some advanced topics in Computer Graphics taught?
A: Advanced topics in computer graphics include three-dimensional rendering, curve and surface design, antialiasing, animation, and visualization.


In [102]:
print(generate_response("How many students does PFW have?"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: How many students does PFW have?
A: PFW has approximately 8,110 students as of fall 2023.
